Используя набор данных IMDB сделано:
1. Спроектировать модель классификации отзывов к фильмам с точностью на валидационной выборке более 80%.
2. Показать, что модель способна классифицировать отзывы с вероятностью более 88% на контрольной выборке.

In [ ]:
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras import utils
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Загрузка данных
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [ ]:
# Вывод размеров полученных наборов данных
print("Размеры тренировочной выборки:", len(train_data))
print("Размеры проверочной выборки:", len(test_data))

In [ ]:
import numpy as np
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension)) # Создаем нулевой вектор с размерностью 10000!
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1. # Записываем единицы в элемент с данным индексом
    return results

x_train = vectorize_sequences(train_data) # Векторизуем обучающие данные
x_test  = vectorize_sequences(test_data)  # Векторизуем контрольные данные

In [ ]:
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

In [ ]:
import keras
from keras.callbacks import EarlyStopping
from keras import models
from keras import layers
from keras import regularizers

# Определим раннюю остановку
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


model = models.Sequential()
model.add(layers.Dense(4096, activation='relu', kernel_regularizer=regularizers.l2(0.02), input_shape=(10000,)))
model.add(layers.Dropout(0.7))
model.add(layers.Dense(2048, activation='relu', kernel_regularizer=regularizers.l2(0.02)))
model.add(layers.Dropout(0.7))
model.add(layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.02)))
model.add(layers.Dropout(0.7))
model.add(layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.02)))
model.add(layers.Dropout(0.7))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001)

EPOCHS = 30
BATCH_SIZE = 256


model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


history = model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE,
                    validation_split=0.1, callbacks=[early_stopping])

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
import matplotlib.pyplot as plt
history_dict = history.history
epochs = range(1, len(history_dict['loss'])+1)
plt.plot(epochs, history_dict['loss'], 'bo', label='Потери на этапе обучения')
plt.plot(epochs, history_dict['val_loss'], 'b', label='Потери на этапе проверки')
plt.title('Потери на этапах проверки и обучения')
plt.xlabel('Эпохи')
plt.ylabel('Потери')
plt.legend()
plt.show()

In [ ]:
plt.clf() # Очистить рисунок
plt.plot(epochs, history_dict['accuracy'], 'bo', label='Точность на этапе обучения')
plt.plot(epochs, history_dict['val_accuracy'], 'b', label='Точность на этапе проверки')
plt.title('Точность на этапе проверки и обучения')
plt.xlabel('Эпохи')
plt.ylabel('Точность')
plt.legend()
plt.show()

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)
print("Доля верных ответов на тестовых данных, в процентах:", round(scores[1] * 100, 4))